# Wrangler Prefect Walkthrough

### Introduction

In this lesson, we'll move through the wrangler prefect lab.  The lab will use the texas drink receipts api. 

### Project Justification

Before moving on, let's talk through why this topic can make for a good project.  We can use alcoholic receipts as a proxy for revenue of a company, and here we can get insights on a per store basis, that we cannot get from a company report.  

For example, in the `etl/tx_drinks/seed` file, you can see that we chose restaurants that are publicly listed.  The last set of restaurants are different Applebees chains.  We could imagine a project showing the following.

* Overview analysis
1. Showing a relationship between drink revenue and overall revenue
2. Master/map display of revenue per store
3. Detail - data about an individual store, as we click on a store
    * Eg. Revenue per month, monthly sales in previous year vs current
    
* Deeper Analysis
And then if given more time, we could explore location based metrics.
1. Does income of region predict revenue?
2. How about population size?
3. Or sales of nearby competitors (eg. Chilis)
4. Outliers - Are there over or underperformers that we should look more deeply at?

Stats like this would allow us to (1) justify where to open/close retail locations of similar stores (2) assess performance to dig deeper and learn about a store's performance.


### Getting setup 

To be able to run the code, we should setup our virtual environment, and install the necessary packages.

```bash
python3 python -m venv ./venv
```

```bash
source venv/bin/activate
```

Now that the environment is activated, we can install the packages in the `requirements.txt` file.

```bash
pip3 install -r requirements.txt
```

And then we should assign the `PYTHONPATH` to the path of our `revenue_tracker` folder.  This will mean that as we import files, it will always relative to the specified revenue_tracker.  

In my environment, I set this by placing the following into bash.

```bash
export PYTHONPATH=/Users/jeffreykatz/Documents/jigsaw/curriculum/1-career-services/prefect-lessons/9-aws-wrangler-lab/revenue_tracker
```

But you should navigate to your `revenue_tracker` folder, and type `pwd` to find the absolute path to your folder.  

Now, when in our code we have something like:

```python
import settings
```

It will look for the module in the `revenue_tracker` folder.

From there, we should set the environmental variables, which you can see in the `.env` file.  As you can see, we need a specified bucket folder to store our data, as well as a name for our glue_db (eg. `revenue_tracker` is fine).

### Kicking off the project

A good place to start with the project is the `console.py` file.  At the bottom, you can find some useful functions to try out.
* `find_all_receipts`
    * `receipts_client.find_all_receipts`
        * Our client pulling down data from the api
    * `receipts_adapter.coerce_df`
        * This function is tricky.  The idea is to use pandas to convert as many columns as possible into either a numeric or datetime column.  This way, when we save the dataframe to athena, it will have those numeric/datetime datatypes.  Notice that we return if the dataframe is empty, which may occur if we do not get records back from the api.
        
* `aws_utils.write_to_s3`
    * this is in the `aws_utils` as there are multiple times that we may want to write to s3, and we wanted to avoid repeating the code.
    
> Choosing the partition key

> To choose the partition, we considered various candidates.  Using an item like obligation_end_date was the first consideration (and what we went with) because it consistently updated each month.  Still, we coerced it into a date -- and perhaps should have only selected month and year -- as too detailed (eg. considering the time, or day) could lead to partition explosion.  We also considered using additional partition keys of say restaurant name (which could make when storing data from multiple restaurants), however we saw there were only 21 locations per an example restaurant, which seemed like to sparse a partition.

```python
df.location_address.unique()
# 21
```

So that's the first major step, `find_and_coerce` the data, and then write it to s3.  One thing to note is that by default, the api only returns 1000 records.  So to seed our database, we would [change the limit](https://dev.socrata.com/docs/paging.html) to the max of `50_000` records.  This would give us an initial dataset, and we could schedule our prefect code to repeatedly update this (more below).

### Setting up Glue/Athena

So after we have seeded our data lake with data, the next step is to tell Glue to crawl it so that we can query the data with a query engine like Athena or Pyspark.

For this, we can continue on at the bottom of the `console.py` file.  

* `athena.migrations.create_db`
    * This creates a database in glue.
    
* `athena.migrations.crawl_dataset("receipts")`
    * This creates a new table in our database called `receipts`, sets the specified datatypes, and seeds with our data.
    * One confusing component, is that because we partitioned by `obligation_end_date`, this will be an object, and not our preferred datetime.  

Still if we run those two functions, we should see our database seeded with some initial data.

* `athena.queries.read_query`

From there, we can query our dataset with athena with our `read_query(query)` function.

### Seeding a list of llcs

Eventually, we would like this to work for a list of llcs.  And we want this to be a common list of llcs regardless of whose computer is running this.

So in the console.py file you can see that we call `write_llcs`, which writes to a parquet file in S3, a list of llc names defined in `seed/restaurant_llcs.py`.  Then we can read them and get back a list of llcs.

### Our Run file

Ok, so now we have seeded and crawled our database with some receipts data.  If you look at the `etl/tx_drinks/run.py` file, you can see that our flow works similarly, but slightly differently from what we just walked through.

The main issue is that when we already have restaurant data in S3, we just want to pull new data, and when it's a new restaurant we want to pull all data.

* find_and_coerce
    * This calls `receipts_client.find`, which first looks for new data, or if we don't have receipts for the restaurant in the db will find all data. 
   * `receipts_client.find_recent()`
        * `queries.find_last_end_date(name)`
            * It does this by first finding the last obligation end_date of the specified restaurant.  
        * `receipts_client.find_receipts_after(name, last_end_date)`
            * Then having been provided that end date, we query the api only for restaurant receipts after that date.
        * `find_all_receipts()`
            * Called if there are no existing receipts. 

* `find_and_coerce_llcs`()
    * calls find_and_coerce for all llcs stored in our s3 file.
    * From there, we coerce the dataframe, and store in s3.

So the above approach will allow us to just query for new receipts that we have not seen before.

### Flow

Now onto the flow.py file.  Notice that once again there is almost **hardly any code** in the flow.py file.

The only exception is the `find_and_write_receipts` flow.  And even that, we have another function that in run.py that essentially does the same thing.  

So again, the point is to test and write as much logic as possible outside of our workflow manager, prefect.  This makes it easier to test, and rapidly speeds up the feedback loop when we try our code, instead of the slow task of waiting for a flow to run.